In [9]:
!pip install flask pyngrok


In [10]:
from pyngrok import ngrok
ngrok.set_auth_token("2xEEC24LdeWtwW1UeS8fETPZGGS_6RNNxrdazCKCKyjrowHv6")


In [ ]:
from flask import Flask, request, render_template_string, redirect, url_for
from pyngrok import ngrok
from datetime import datetime
from collections import defaultdict
import json
import os


DATA_FILE = "data.json"



try:
    public_url = ngrok.connect(5000).public_url
except Exception as e:
    print(f"Failed to connect to ngrok: {e}")
    public_url = "Ngrok connection failed."

print(" * Public URL:", public_url)


policyholders = {}
claims = []

class Policyholder:
    counter = 1
    def __init__(self, name, age, policy_type, sum_insured):
        self.id = Policyholder.counter
        Policyholder.counter += 1
        self.name = name
        self.age = age
        self.policy_type = policy_type
        self.sum_insured = sum_insured
        self.claims = []

class Claim:
    counter = 1
    def __init__(self, policyholder_id, amount, reason, date_of_claim):
        self.claim_id = Claim.counter
        Claim.counter += 1
        self.policyholder_id = policyholder_id
        self.amount = amount
        self.reason = reason
        self.status = "Pending"
        self.date_of_claim = datetime.strptime(date_of_claim, '%Y-%m-%d')



def load_data():
    global policyholders, claims
    if not os.path.exists(DATA_FILE):
        return
    with open(DATA_FILE, "r") as f:
        data = json.load(f)

    policyholders.clear()
    for p in data.get("policyholders", []):
        ph = Policyholder(p['name'], p['age'], p['policy_type'], p['sum_insured'])
        ph.id = p['id']
        ph.claims = []
        policyholders[ph.id] = ph
    Policyholder.counter = max(policyholders.keys(), default=0) + 1

    claims.clear()
    for c in data.get("claims", []):
        claim = Claim(c['policyholder_id'], c['amount'], c['reason'], c['date_of_claim'])
        claim.claim_id = c['claim_id']
        claim.status = c['status']
        claims.append(claim)
        if claim.policyholder_id in policyholders:
            policyholders[claim.policyholder_id].claims.append(claim)
    Claim.counter = max((c.claim_id for c in claims), default=0) + 1

def save_data():
    data = {
        "policyholders": [
            {
                "id": ph.id,
                "name": ph.name,
                "age": ph.age,
                "policy_type": ph.policy_type,
                "sum_insured": ph.sum_insured,
            } for ph in policyholders.values()
        ],
        "claims": [
            {
                "claim_id": c.claim_id,
                "policyholder_id": c.policyholder_id,
                "amount": c.amount,
                "reason": c.reason,
                "status": c.status,
                "date_of_claim": c.date_of_claim.strftime("%Y-%m-%d")
            } for c in claims
        ]
    }
    with open(DATA_FILE, "w") as f:
        json.dump(data, f, indent=4)


app = Flask(__name__)

BOOTSTRAP_HEADER = """
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
<div class='container mt-5'>
<h2 class='mb-4 text-center text-primary'>Insurance Claims Management System</h2>
"""

@app.route('/')
def home():
    return render_template_string(BOOTSTRAP_HEADER + """
    <div class="list-group">
      <a href="/register" class="list-group-item list-group-item-action">Register Policyholder</a>
      <a href="/claim" class="list-group-item list-group-item-action">Add Claim</a>
      <a href="/reports" class="list-group-item list-group-item-action">View Reports</a>
      <a href="/policyholders" class="list-group-item list-group-item-action">View All Policyholders</a>
      <a href="/public_url" class="list-group-item list-group-item-action">Show Ngrok Public URL</a>
    </div>
    </div>
    """)

@app.route('/public_url')
def show_public_url():
    return render_template_string(BOOTSTRAP_HEADER + f"""
        <h4>Current Ngrok Public URL:</h4>
        <div class="alert alert-info">
            <a href="{public_url}" target="_blank">{public_url}</a>
        </div>
        <a class="btn btn-secondary" href="/">Back</a>
    </div>
    """)

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        name = request.form['name']
        age = int(request.form['age'])
        policy_type = request.form['policy_type']
        sum_insured = float(request.form['sum_insured'])
        ph = Policyholder(name, age, policy_type, sum_insured)
        policyholders[ph.id] = ph
        save_data()
        return redirect(url_for('home'))
    return render_template_string(BOOTSTRAP_HEADER + """
    <h4>Register New Policyholder</h4>
    <form method='post' class="row g-3">
        <div class="col-md-6">
            <label>Name *</label>
            <input name='name' class="form-control" required>
        </div>
        <div class="col-md-6">
            <label>Age *</label>
            <input name='age' type='number' class="form-control" required min="1">
        </div>
        <div class="col-md-6">
            <label>Policy Type *</label>
            <select name='policy_type' class="form-select" required>
                <option value="">Select</option>
                <option>Health</option>
                <option>Vehicle</option>
                <option>Life</option>
            </select>
        </div>
        <div class="col-md-6">
            <label>Sum Insured *</label>
            <input name='sum_insured' type='number' class="form-control" step="0.01" required min="1">
        </div>
        <div class="col-12">
            <button type='submit' class="btn btn-success">Register</button>
            <a href="/" class="btn btn-secondary">Cancel</a>
        </div>
    </form>
    </div>
    """)

@app.route('/claim', methods=['GET', 'POST'])
def claim():
    if request.method == 'POST':
        policyholder_id = int(request.form['policyholder_id'])
        amount = float(request.form['amount'])
        reason = request.form['reason']
        date_of_claim = request.form['date_of_claim']
        if policyholder_id not in policyholders:
            return "Invalid Policyholder ID"
        c = Claim(policyholder_id, amount, reason, date_of_claim)
        claims.append(c)
        policyholders[policyholder_id].claims.append(c)
        save_data()
        return redirect(url_for('home'))
    return render_template_string(BOOTSTRAP_HEADER + """
    <h4>Add Claim</h4>
    <form method='post' class="row g-3">
        <div class="col-md-6">
            <label>Policyholder ID *</label>
            <input name='policyholder_id' type='number' class="form-control" required>
        </div>
        <div class="col-md-6">
            <label>Amount *</label>
            <input name='amount' type='number' class="form-control" step="0.01" required min="1">
        </div>
        <div class="col-md-6">
            <label>Reason *</label>
            <input name='reason' class="form-control" required>
        </div>
        <div class="col-md-6">
            <label>Date of Claim (YYYY-MM-DD) *</label>
            <input name='date_of_claim' type="date" class="form-control" required>
        </div>
        <div class="col-12">
            <button type='submit' class="btn btn-warning">Add Claim</button>
            <a href="/" class="btn btn-secondary">Cancel</a>
        </div>
    </form>
    </div>
    """)

@app.route('/policyholders')
def list_policyholders():
    return render_template_string(BOOTSTRAP_HEADER + """
    <h4>Registered Policyholders</h4>
    <ul class="list-group">
    {% for ph in policyholders.values() %}
        <li class="list-group-item">{{ ph.name }} (ID: {{ ph.id }}) -
            <a href="{{ url_for('view_policyholder', id=ph.id) }}">View Details</a></li>
    {% else %}
        <li class="list-group-item">No policyholders registered.</li>
    {% endfor %}
    </ul>
    <a href="/" class="btn btn-secondary mt-3">Back</a>
    </div>
    """, policyholders=policyholders)

@app.route('/policyholder/<int:id>')
def view_policyholder(id):
    ph = policyholders.get(id)
    if not ph:
        return f"Policyholder with ID {id} not found.", 404
    return render_template_string(BOOTSTRAP_HEADER + """
    <h4>Policyholder Details</h4>
    <p><strong>ID:</strong> {{ ph.id }}</p>
    <p><strong>Name:</strong> {{ ph.name }}</p>
    <p><strong>Age:</strong> {{ ph.age }}</p>
    <p><strong>Policy Type:</strong> {{ ph.policy_type }}</p>
    <p><strong>Sum Insured:</strong> ₹{{ '%.2f'|format(ph.sum_insured) }}</p>

    <h5 class="mt-3">Claims:</h5>
    {% if ph.claims %}
        <ul class="list-group">
        {% for c in ph.claims %}
            <li class="list-group-item">Claim ID: {{ c.claim_id }}, Amount: ₹{{ '%.2f'|format(c.amount) }},
            Reason: {{ c.reason }}, Date: {{ c.date_of_claim.strftime('%Y-%m-%d') }},
            Status: {{ c.status }}</li>
        {% endfor %}
        </ul>
    {% else %}
        <p>No claims found for this policyholder.</p>
    {% endif %}
    <a href="{{ url_for('list_policyholders') }}" class="btn btn-secondary mt-3">Back</a>
    </div>
    """, ph=ph)

@app.route('/reports')
def reports():
    high_risk = []
    claim_frequency = {}
    for ph in policyholders.values():
        claim_frequency[ph.id] = len(ph.claims)
        if not ph.claims:
            continue
        recent_claims = [c for c in ph.claims if (datetime.now() - c.date_of_claim).days <= 365]
        total_claim_amount = sum(c.amount for c in ph.claims)
        claim_ratio = total_claim_amount / ph.sum_insured if ph.sum_insured > 0 else 0
        if len(recent_claims) > 3 or claim_ratio > 0.8:
            high_risk.append(ph)

    claim_types = defaultdict(int)
    for c in claims:
        if c.policyholder_id in policyholders:
            ph = policyholders[c.policyholder_id]
            claim_types[ph.policy_type] += 1

    monthly_claims = defaultdict(int)
    for c in claims:
        key = c.date_of_claim.strftime('%Y-%m')
        monthly_claims[key] += 1

    totals_by_type = defaultdict(list)
    for c in claims:
        if c.policyholder_id in policyholders:
            ph = policyholders[c.policyholder_id]
            totals_by_type[ph.policy_type].append(c.amount)
    avg_claim_amount = {k: sum(v)/len(v) if v else 0 for k, v in totals_by_type.items()}

    highest_claim = max(claims, key=lambda c: c.amount) if claims else None
    pending_claims = [c for c in claims if c.status == "Pending"]

    return render_template_string(BOOTSTRAP_HEADER + """
    <h4>Reports</h4>

    <h5>High-Risk Policyholders:</h5>
    <ul>{% for h in high_risk %}<li>{{ h.name }} (ID: {{ h.id }})</li>{% endfor %}</ul>

    <h5>Claim Frequency per Policyholder:</h5>
    <ul>{% for k, v in claim_frequency.items() %}
        <li>{{ policyholders[k].name }} (ID: {{ k }}): {{ v }} claims</li>{% endfor %}
    </ul>

    <h5>Claims by Policy Type:</h5>
    <ul>{% for k, v in claim_types.items() %}<li>{{ k }}: {{ v }}</li>{% endfor %}</ul>

    <h5>Total Claims Per Month:</h5>
    <ul>{% for k, v in monthly_claims.items() %}<li>{{ k }}: {{ v }}</li>{% endfor %}</ul>

    <h5>Average Claim Amount by Policy Type:</h5>
    <ul>{% for k, v in avg_claim_amount.items() %}<li>{{ k }}: ₹{{ '%.2f'|format(v) }}</li>{% endfor %}</ul>

    <h5>Highest Claim Filed:</h5>
    {% if highest_claim %}
        <p>₹{{ highest_claim.amount }} by {{ policyholders[highest_claim.policyholder_id].name }}</p>
    {% else %}
        <p>No claims available.</p>
    {% endif %}

    <h5>Pending Claims:</h5>
    {% if pending_claims %}
        <ul>{% for p in pending_claims %}
            <li>{{ policyholders[p.policyholder_id].name }} - ₹{{ p.amount }} (ID: {{ p.claim_id }})</li>{% endfor %}
        </ul>
    {% else %}
        <p>No pending claims.</p>
    {% endif %}
    <a href='/' class='btn btn-secondary mt-3'>Back</a>
    </div>
    """, high_risk=high_risk, claim_frequency=claim_frequency,
         claim_types=claim_types, monthly_claims=monthly_claims,
         avg_claim_amount=avg_claim_amount, highest_claim=highest_claim,
         pending_claims=pending_claims, policyholders=policyholders)


if __name__ == '__main__':
    load_data()
    print("Flask server starting...")
    app.run(port=5000)
